### Optimal Tactical Pathing for UGV

In [8]:
import gurobipy as gp
from gurobipy import Model, GRB
import numpy as np
import scipy.sparse as sp
import pandas as pd
import os
import csv
from PIL import Image

In [9]:
SITUATION = "Buckner"
SPACING = 40

In [10]:
sat_path = os.path.join(os.getcwd(), 'imagery', SITUATION+'.png')
sat_map = np.asarray(Image.open(sat_path))
l, w, h = sat_map.shape
rows = l // SPACING
cols = w // SPACING
cols


3

In [11]:
arcs = pd.read_csv('Buckner_arcs_3_3.csv',header=None)
arcs
ins_csv = pd.read_csv('Buckner_ins_3_3.csv', header=None)
outs_csv = pd.read_csv('Buckner_outs_3_3.csv', header=None)

arcs


,0,1,2,3,4,5,6
0,1,1,2,0.005661,179.104478,-0.443756,0
1,2,1,5,1.000000,179.104478,232.573500,0
2,3,1,4,0.005316,179.104478,352.700609,0
3,4,1,10,0.050055,179.104478,0.000000,1
4,5,2,3,0.005152,179.104478,260.180215,0
...,...,...,...,...,...,...,...
93,94,17,8,0.005866,179.104478,0.000000,0
94,95,18,17,0.047182,179.104478,-111.114877,1
95,96,18,14,1.000000,179.104478,-57.404831,1
96,97,18,15,0.051187,179.104478,-115.663070,1


In [12]:
inflow = ins_csv.apply(lambda row: row[row != 0].tolist(), axis=1).to_dict()
inflow = {
    idx + 1: row[row != 0].tolist()
    for idx, row in ins_csv.iterrows()
}


outflow = outs_csv.apply(lambda row: row[row != 0].tolist(), axis=1).to_dict()
outflow = {
    idx + 1: row[row != 0].tolist()
    for idx, row in outs_csv.iterrows()
}

In [13]:
inflow

{1: [9, 18, 26, 53],
 2: [1, 13, 19, 27, 33, 59],
 3: [5, 28, 34, 63],
 4: [3, 8, 25, 37, 42, 69],
 5: [2, 7, 12, 15, 32, 38, 43, 47, 78],
 6: [6, 11, 21, 44, 48, 84],
 7: [17, 24, 41, 88],
 8: [16, 23, 31, 36, 46, 94],
 9: [22, 30, 40, 98],
 10: [4, 58, 67, 75],
 11: [10, 50, 62, 68, 76, 82],
 12: [14, 54, 77, 83],
 13: [20, 52, 57, 74, 86, 91],
 14: [29, 51, 56, 61, 64, 81, 87, 92, 96],
 15: [35, 55, 60, 70, 93, 97],
 16: [39, 66, 73, 90],
 17: [45, 65, 72, 80, 85, 95],
 18: [49, 71, 79, 89]}

In [14]:
t = 60*60*60
s = 1
f = 9
N=cols*rows*2
A=max(arcs[0])
d = arcs[3]
E = arcs[5]
t = 60*60*60

batteryCapacity = 100000
A

98

In [23]:
if isinstance(A, int):  
    A = list(range(1, A+1))

In [55]:
from gurobipy import Model, GRB, quicksum

# Define the nodes, arcs, and time cost
nodes = [1, 2, 3, 4, 5]
arcs = [(1, 2), (1, 3), (2, 4), (3, 4), (4, 5)]
time_cost = {(1, 2): 4, (1, 3): 2, (2, 4): 3, (3, 4): 1, (4, 5): 5}

start_node = 1
end_node = 5

# Create Gurobi model
model = Model("Shortest Path")

# Decision variables: whether an arc is in the path
x = model.addVars(arcs, vtype=GRB.BINARY, name="x")

# Objective: minimize total time cost
model.setObjective(quicksum(time_cost[a] * x[a] for a in arcs), GRB.MINIMIZE)

# Flow constraints: ensure path continuity
for node in nodes:
    if node == start_node:
        model.addConstr(quicksum(x[i, j] for i, j in arcs if i == node) - 
                        quicksum(x[j, i] for j, i in arcs if i == node) == 1)
    elif node == end_node:
        model.addConstr(quicksum(x[i, j] for i, j in arcs if i == node) - 
                        quicksum(x[j, i] for j, i in arcs if i == node) == -1)
    else:
        model.addConstr(quicksum(x[i, j] for i, j in arcs if i == node) - 
                        quicksum(x[j, i] for j, i in arcs if i == node) == 0)

# Solve the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print("Optimal path:")
    for a in arcs:
        if x[a].x > 0.5:
            print(f"{a} -> Time: {time_cost[a]}")
    print(f"Total Time: {model.objVal}")
else:
    print("No optimal solution found.")




def write_path(opt_path, scenario_name="3x3"):
    path_df = pd.DataFrame(opt_path, columns=["Path"])
    path_df.to_csv(f"output_{scenario_name}.csv", index=False)
    print(f"Path saved to output_{scenario_name}.csv")


opt_path = simple_path_solver(A, N, inflow, outflow)
write_path(opt_path)

print(opt_path)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 5 columns and 10 nonzeros
Model fingerprint: 0x3a99393a
Variable types: 0 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 5 rows and 5 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 8 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.000000000000e+00, best bound 8.000000000000e+00, gap 0.0000%
Optimal path:
(1, 3) -> Time: 2
(3, 4) -> Time: 1
(4, 5) -> Time: 5
Total Time: 8.0
Node 0 inflow: []
Nod

Set parameter TimeLimit to value 120
Set parameter MIPGap to value 0.01


TypeError: 'list' object cannot be interpreted as an integer